In [11]:
from __future__ import print_function
import numpy as np
import pandas as pd
import scipy
import sys
import os
from time import time
np.random.seed(1234567)

sys.path.append('../')
sys.path.append('../../')
sys.path.append('/Users/adam2392/Documents/tvb/')
sys.path.append('/Users/adam2392/Documents/tvb/_tvbdata/')
sys.path.append('/Users/adam2392/Documents/tvb/_tvblibrary/')
import processing.preprocessfft as preprocess
import processing.frequencytransform as ft

sys.path.append('/Users/adam2392/Documents/fragility_analysis/')
import fragility
import fragility.util.utils as futil
import datainterface
from datainterface import patient as Pat


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Convert Data

Data will be stored either as edf files, or .eeg files. We can use our eegreader to read .eeg files (if no bugs) and pyedflib to read the .edf files.

Then, we want to make sure we have the vital information regarding each dataset.

In [9]:
filename = '/Users/adam2392/Downloads/tngrawdata/id001_ac/crise1/crise cabrol.eeg'
eegreader = datainterface.read_eeg.EEG(filename)


reading header from <open file '/Users/adam2392/Downloads/tngrawdata/id001_ac/crise1/crise cabrol.eeg', mode 'rb' at 0x113dc8f60>


IOError: <open file '/Users/adam2392/Downloads/tngrawdata/id001_ac/crise1/crise cabrol.eeg', mode 'rb' at 0x113dc8f60> is not a valid EEG file

In [19]:
#### 1. first run data conversion
patient = 'id001_ac'
outputdir = os.path.join('/Volumes/ADAM LI/pydata/convertedtng/', patient)
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

edfdir = os.path.join('/Users/adam2392/Downloads/tngrawdata/', patient)
edffile = os.path.join(edfdir, '')
edffile ='/Users/adam2392/Downloads/tngrawdata/id001_ac/crise2/crise_fin/091126B-BEX_0006.edf'

In [20]:
#### 1. setting filename 
npyfile = os.path.join(outputdir,  patient.lower() + '_rawnpy.npy')
chanfile =  os.path.join(outputdir,  patient.lower() + '_chans.csv')
headerfile = os.path.join(outputdir,  patient.lower() + '_headers.csv')
annotationsfile = os.path.join(outputdir, patient.lower() + '_annotations.csv')

# convert data into csv files and numpy raw data file
converter = datainterface.dataconversion.EDFConverter(edffile)
converter.edfrawtonumpy(npyfile)
converter.edfmetatocsv(headerfile, \
                    chanfile, \
                    annotationsfile)
f1=open(os.path.join(outputdir, 'datadescrip.txt'), 'w+')
f1.write(edffile)
f1.close()

INFO:datainterface.dataconversion:Initialized EDFConverter object.
INFO:datainterface.dataconversion:File is edf format: /Users/adam2392/Downloads/tngrawdata/id001_ac/crise2/crise_fin/091126B-BEX_0006.edf
INFO:datainterface.dataconversion:Closing edf file! Raw data should be saved as numpy at /Volumes/ADAM LI/pydata/convertedtng/id001_ac/id001_ac_rawnpy.npy
INFO:datainterface.dataconversion:Closing edf file! Headers meta data should be saved as csv at /Volumes/ADAM LI/pydata/convertedtng/id001_ac/id001_ac_headers.csv
INFO:datainterface.dataconversion:Channel meta data should be saved as csv at /Volumes/ADAM LI/pydata/convertedtng/id001_ac/id001_ac_chans.csv
INFO:datainterface.dataconversion:Annotations meta data should be saved as csv at /Volumes/ADAM LI/pydata/convertedtng/id001_ac/id001_ac_annotations.csv


In [13]:
expname = 'id001_ac'
traindir = os.path.join('/Volumes/ADAM LI/pydata/testdata/fft/', 
                        expname)
if not os.path.exists(traindir):
    os.makedirs(traindir)

# establish frequency bands
freqbands = {
        'dalpha':[0,15],
        'beta':[15,30],
        'gamma':[30,90],
        'high':[90,200],
    }
postprocessfft = preprocess.PreProcess(freqbands=freqbands)
# FFT Parameters
fs = 1000
winsize = 1000 # winsize in milliseconds
stepsize = 500 # stepsize in milliseconds
typetransform = 'fourier'
mtbandwidth = 4
mtfreqs = []

mtaper = ft.MultiTaperFFT(winsize, stepsize, fs, mtbandwidth, mtfreqs)

print(traindir)

Default method of tapering is eigen
/Volumes/ADAM LI/pydata/testdata/fft/id001_ac


# 1. Run FFT Analysis 

In [12]:
def extractrawdata(patient, datadir, fileend=None):
    '''
    This function takes in:
    1. patient name
    2. results directory (perturbation model)
    3. data directory, where all the raw metadata is held
    
    And extracts the data necessary for plotting
    '''
    # files to get resulting data and the original raw data / annotations
    rawdatafile = os.path.join(datadir,  patient + fileend, patient+'_rawnpy.npy')
    chanfile =  os.path.join(datadir,  patient + fileend, patient + '_chans.csv')
    headerfile = os.path.join(datadir,  patient + fileend, patient + '_headers.csv')
    annotationsfile = os.path.join(datadir,  patient + fileend, patient + '_annotations.csv')

    ################################ 1. LOAD DATA ###########################
    # instantiate a datainterface object to analyze data
    patieeg = Pat.PatientIEEG(patient, clinoutcome=None, engelscore=None, logfile=None)
    # get relevant channel data
    patid, seizid = futil.splitpatient(patient)
    included_chans, onsetchans, clinresult = fragility.util.utils.returnindices(patid, seizid)
    
    # set metadata to be used in analysis
    patieeg.setincludedchans(included_chans)
    patieeg.setmetadata_fromfile(headersfile=headerfile)
    patieeg.setchannels_fromfile(channelsfile=chanfile)
    patieeg.setannotations_fromfile(annotationsfile=annotationsfile)

    # get the data from files
    data = np.load(rawdatafile)
    
    return patieeg, data, included_chans, onsetchans

In [5]:
datadir = '/Volumes/ADAM LI/pydata/convertedtng/'
patient='id001_ac'
# load in the data for a real patient
patieeg, rawdata, included_chans, onsetchans = extractrawdata(patient, datadir, '_sz2')

rawdata=rawdata.T
print(patieeg)
print(rawdata.shape)
print(included_chans.shape)
print(onsetchans)

INFO:datainterface.patient:Initialized Patient IEEG object. Should run channels, metadata and annotations next!
INFO:datainterface.patient:Ran setup of meta data!
INFO:datainterface.patient:Ran setup of channels data!
INFO:datainterface.patient:Ran setup of annotations data!


(84, 306688)
(0,)
None


In [14]:
metadatadir = os.path.join('/Volumes/ADAM LI/pydata/metadata/', patient)  
# load in seegxyz
seeg = pd.read_csv(os.path.join(metadatadir, "seeg.txt"), names=['x', 'y', 'z'], delim_whitespace=True)
chanlabels = seeg.index
seeg_xyz = seeg.as_matrix() 
    
# run FFT analysis
mtaper.loadrawdata(rawdata)
power, freqs, timepoints, _ = mtaper.mtwelch()
power = postprocessfft.binFrequencyValues(power, freqs)

onsettime = patieeg.onset_time
offsettime = patieeg.offset_time
# get seiztimes given onset/offset
if onsettime: 
    onset_indice = fragility.util.utils.converttimestowindow(timepoints, onsettime * fs)
if offsettime:
    offset_indice = fragility.util.utils.converttimestowindow(timepoints, offsettime * fs)
seiztimes = zip(onset_indice, offset_indice)
    
filename = os.path.join(traindir, 
                        patient + '_fft.npz')

# load xyz data for this particular dataset
xyz_data = data['locs']
seeg_contacts = data['seeg_contacts']
x0ez = data['x0ez']
seizonsets = data['seizonsets']
seizoffsets = data['seizoffsets']


np.savez_compressed(filename, 
                    power=power, 
                    timepoints=timepoints,
                    seiztimes=seiztimes,
                    locs=seeg_xyz,
                    seeg_contacts=chanlabels)
print(filename)
print(power.shape)
print(freqs.shape)
print(timepoints.shape)
print('\n\n')

Loaded raw data in MultiTaperFFT!


/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
../processing/preprocessfft.py:80: ComplexWarning: Casting complex values to real discards the imaginary part
  power_binned[:,idx,:] = np.mean(power[:,indices[0]:indices[1]+1,:], axis=1) #[np.newaxis,:,:]


NameError: name 'onset_indice' is not defined

# 2. Run Image Creation for 2D Mesh

In [ ]:
expname = 'varydistance_2'
datadir = os.path.join('/Volumes/ADAM LI/pydata/dnn/traindata/fft/', expname) 

# load in meta data
project_dir = os.path.join(metadatadir, patient)
confile = os.path.join(project_dir, "connectivity.zip")

# get the regions, and region_centers from connectivity
reader = util.ZipReader(confile)
region_centers = reader.read_array_from_file("centres", use_cols=(1, 2, 3))
regions = reader.read_array_from_file("centres", dtype=np.str, use_cols=(0,))



# fftdatafile, seeg contacts, 
datafile = 


In [ ]:
# define the data handler 
datahandler = util.DataHandler()
pca = PCA(n_components=2)
trainimagedir = os.path.join('/Volumes/ADAM LI/pydata/dnn/testdata/image_2d/', expname)
if not os.path.exists(trainimagedir):
    os.makedirs(trainimagedir)
    
# loop through each data file and get grid
data = np.load(datafile, encoding='bytes')
power = data['power']
print(power.shape)
print(data.keys())

# load xyz data for this particular dataset
xyz_data = data['locs']
seeg_contacts = data['seeg_contacts']
x0ez = data['x0ez']
seizonsets = data['seizonsets']
seizoffsets = data['seizoffsets']
timepoints = data['timepoints']

# project xyz data
if AZIM==1:
    print("using azim projection to grid image")
    new_locs = []
    for ichan in range(0,xyz_data.shape[0]):
        new_locs.append(datahandler.azim_proj(xyz_data[ichan,:]))
    new_locs = np.asarray(new_locs)
if AZIM==0:
    print("using pca to grid image")
    new_locs = pca.fit_transform(xyz_data)

print(seizonsets)
print(seizoffsets)
ylabels = datahandler.computelabels(seizonsets,seizoffsets, timepoints)

if len(seizonsets) <= 1 and seizonsets[0] == np.nan:
    print("skipping ", datafile)
else:
    # Tensor of size [samples, freqbands, W, H] containing generated images.
    image_tensor = datahandler.gen_images(new_locs, power, 
                            n_gridpoints=32, normalize=True, augment=False, 
                            pca=False, std_mult=0.1, edgeless=False)

print(image_tensor.shape)
# set saving file paths for image and corresponding meta data
filename = path_leaf(datafile)
imagefilename = os.path.join(trainimagedir, filename)

# instantiate metadata hash table
metadata = dict()
metadata['x0ez'] = x0ez
metadata['seeg_contacts'] = seeg_contacts
metadata['new_locs'] = new_locs
metadata['ylabels'] = ylabels

# save image and meta data
np.savez(imagefilename, image_tensor=image_tensor, metadata=metadata)

print(new_locs.shape)